## Transform ResStock Run Results to 15 SF prototypes (Elevate Energy)

In [1]:
import pandas as pd
import numpy as np
import os
print(f'Notebook path: {os.getcwd()}')

Notebook path: /Users/lliu2/Documents/GitHub/OpenStudio-BuildStock/files


### Download results from S3:
://S3/resbldg-datasets/chicagoeui

In [172]:
# local path to downloaded results
iteration = '_national_2018' # <----- options: 'base', '_01'~'_10', '_national_2018'
result_path = '/Users/lliu2/Documents/Chicago retrofits/ResStock results/'
iteration_path = f'cookcnty_2007{iteration}/baseline' 
baseline_result = os.path.join(result_path,iteration_path,'results_up00.parquet')
print(f'Result and file storage path: {iteration_path}')


Result and file storage path: cookcnty_2007_national_2018/baseline


In [173]:
res = pd.read_parquet(baseline_result)
res

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_maximum_daily_use_overlap_kw,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw
0,1,182,2020-11-20 17:04:00,2020-11-20 17:05:28,Success,NaN,NaN,,Non-CBSA South Atlantic,True,...,12.995735,2.426156,1.717261,1.415638,12.545455,16.090909,25.559785,24.438544,3352.0,31.870529
1,2,30,2020-11-20 13:08:38,2020-11-20 13:10:10,Success,NaN,NaN,,Non-CBSA Mountain,True,...,20.035914,4.332747,4.137072,4.300910,12.727273,14.454545,30.670425,46.062958,688.0,49.567759
2,3,136,2020-11-20 16:54:15,2020-11-20 16:55:43,Success,NaN,NaN,,"CBSA Dallas-Fort Worth-Arlington, TX",True,...,13.554285,3.479243,2.601523,2.113269,16.000000,16.636364,44.463937,29.337732,5127.0,53.732953
3,4,53,2020-11-20 13:00:35,2020-11-20 13:02:02,Success,NaN,NaN,,Non-CBSA West South Central,True,...,6.772384,4.020120,2.556242,1.784218,15.454545,15.363636,25.242941,9.102812,5582.0,29.077394
4,5,32,2020-11-20 11:41:25,2020-11-20 11:42:59,Success,NaN,NaN,,Non-CBSA West North Central,True,...,4.151260,0.757357,0.448335,0.443036,13.181818,8.272727,8.194982,21.020703,293.0,22.238378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549995,549996,192,2020-11-20 17:03:02,2020-11-20 17:05:42,Success,NaN,NaN,,Non-CBSA West North Central,True,...,12.804781,2.975331,1.686071,1.855908,16.000000,16.909091,27.525155,20.352414,3812.0,30.467649
549996,549997,152,2020-11-20 11:39:43,2020-11-20 11:42:32,Success,NaN,NaN,,"CBSA New York-Newark-Jersey City, NY-NJ-PA",True,...,4.311219,1.288380,1.220332,1.269257,14.000000,13.909091,11.556158,11.233843,4629.0,15.127708
549997,549998,170,2020-11-20 14:43:36,2020-11-20 15:09:33,Success,NaN,NaN,,Non-CBSA Pacific,True,...,100.868638,32.038163,31.538810,30.179011,14.454545,11.545455,106.723181,148.384852,8720.0,164.160214
549998,549999,103,2020-11-20 17:31:28,2020-11-20 17:34:08,Success,NaN,NaN,,Non-CBSA East South Central,True,...,9.724379,0.990615,7.204116,1.262223,15.000000,6.272727,10.753170,43.840735,384.0,49.620046


In [174]:
if iteration == '_national_2018':
    res = res[res['build_existing_model.ahs_region']=='CBSA Chicago-Naperville-Elgin, IL-IN-WI'].reset_index(drop=True)
    res

In [175]:
### HVAC cols changed after HVAC restructuring (implemented in Run5)
if iteration in ['base','_01','_02','_03','_04']:
    cooling_col = 'build_existing_model.hvac_system_cooling_type'
else:
    cooling_col = 'build_existing_model.hvac_cooling_type'
if iteration in ['base','_01','_02','_03','_04']:
    heating_col = 'build_existing_model.hvac_system_heating_natural_gas'
else:
    heating_col = 'build_existing_model.hvac_heating_type_and_fuel'
print('selected')


selected


### Check housing charateristics distributions

In [176]:
proto_chars = ['build_existing_model.geometry_stories',
              'build_existing_model.geometry_wall_type',
              'build_existing_model.vintage_acs',
              cooling_col,
              heating_col,
              'build_existing_model.geometry_floor_area'
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'>>> ResStock - {iteration_path} - RESULT SUMMARY:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['building_id'].count()
    Nchar = Nchar/Ns
    Nchar['N_failed'] = len(res[res[char].isnull()])
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(os.path.join(result_path, iteration_path, 'baseline_housing_char_breakdown.csv'), index=False)
print(f'>>> file saved to:\n  {os.path.join(result_path, iteration_path)}')

>>> ResStock - cookcnty_2007_national_2018/baseline - RESULT SUMMARY:

  * 15707 / 15707 samples ran successfully, 0 failed, efficacy: 100.0% 

>>> Housing characteristics splits:

  * [1] build_existing_model.geometry_stories
1           0.507035
2           0.338830
3           0.154135
N_failed    0.000000
Name: building_id, dtype: float64

  * [2] build_existing_model.geometry_wall_type
Masonry     0.441205
WoodStud    0.558795
N_failed    0.000000
Name: building_id, dtype: float64

  * [3] build_existing_model.vintage_acs
1940-59     0.195200
1960-79     0.262685
1980-99     0.204877
2000-09     0.117273
2010s       0.011969
<1940       0.207996
N_failed    0.000000
Name: building_id, dtype: float64

  * [4] build_existing_model.hvac_cooling_type
Central AC    0.507799
Heat Pump     0.020500
None          0.164003
Room AC       0.307697
N_failed      0.000000
Name: building_id, dtype: float64

  * [5] build_existing_model.hvac_heating_type_and_fuel
Electricity ASHP                

### Add additional metrics to ResStock results

In [177]:
# (1) add sqft and eui
sqft_map = {'0-499': 328,
            '500-749': 633,
            '750-999': 885,
            '1000-1499': 1220,
            '1500-1999': 1690,
            '2000-2499': 2176,
            '2500-2999': 2663,
            '3000-3999': 3301,
            '4000+': 8194}

res['sqft'] = res['build_existing_model.geometry_floor_area'].map(sqft_map)
res['gas_eui'] = res['simulation_output_report.total_site_natural_gas_therm'].divide(res['sqft'])
res['elec_eui'] = res['simulation_output_report.total_site_electricity_kwh'].divide(res['sqft'])


# (2) aggregate heating and cooling end uses in ResStock results
res['total_gas_heating'] = res['simulation_output_report.natural_gas_central_system_heating_therm'] +\
    res['simulation_output_report.natural_gas_heating_therm']

res['total_elec_heating'] = res['simulation_output_report.electricity_central_system_heating_kwh'] +\
    res['simulation_output_report.electricity_central_system_pumps_heating_kwh'] +\
    res['simulation_output_report.electricity_fans_heating_kwh'] +\
    res['simulation_output_report.electricity_heating_kwh'] +\
    res['simulation_output_report.electricity_pumps_heating_kwh']

res['total_elec_cooling'] = res['simulation_output_report.electricity_central_system_cooling_kwh'] +\
    res['simulation_output_report.electricity_central_system_pumps_cooling_kwh'] +\
    res['simulation_output_report.electricity_cooling_kwh'] +\
    res['simulation_output_report.electricity_fans_cooling_kwh'] +\
    res['simulation_output_report.electricity_pumps_cooling_kwh']

## EE prototype tags
res['vintage_ee'] = 'post-1978'
res.loc[res['build_existing_model.vintage_acs'].isin(['1940-59','1960-79']),'vintage_ee'] = '1942-1978'
res.loc[res['build_existing_model.vintage_acs']=='<1940','vintage_ee'] = 'pre-1942'

res['stories'] = '2 or more stories'
res.loc[res['build_existing_model.geometry_stories']=='1','stories'] = 'less than 2 stories'

print('new cols added to "res"')

new cols added to "res"


In [148]:
### get sqft distr by building type and vintage
res_sqft_distr = res.groupby(['build_existing_model.geometry_building_type_recs', 'build_existing_model.vintage_acs',
    'build_existing_model.geometry_floor_area'])['building_id'].count().to_frame()

# normalize
res_sqft_distr = res_sqft_distr.groupby(level = [0,1])['building_id'].apply(lambda x: x/x.sum()).unstack(level=2)

# export
res_sqft_distr.to_csv(os.path.join(result_path, iteration_path, 'building_type_vintage_floor_area.csv'), index=True)
print('sqft distr by vintage & bldg type exported')


sqft distr by vintage & bldg type exported


In [97]:
### get foundation type distr by building type, vintage

res_found_distr = res[res['build_existing_model.geometry_building_type_recs'] =="Single-Family Detached"].groupby(
    ['build_existing_model.vintage_acs', 'build_existing_model.geometry_stories',
    'build_existing_model.geometry_foundation_type'])['building_id'].count().to_frame()

# normalize
res_found_distr = res_found_distr.groupby(level = [0,1])['building_id'].apply(lambda x: x/x.sum()).unstack(level=2)
res_found_distr



build_existing_model.geometry_foundation_type                              Crawl  \
build_existing_model.vintage_acs build_existing_model.geometry_stories             
1940-59                          1                                      0.126927   
                                 2                                      0.134772   
                                 3                                      0.147059   
1960-79                          1                                      0.158642   
                                 2                                      0.164084   
                                 3                                      0.164179   
1980-99                          1                                      0.192175   
                                 2                                      0.209333   
                                 3                                      0.206897   
2000-09                          1                                      0.117834   
                                 2                                      0.088517   
                                 3                                      0.100000   
2010s                            1                                      0.071429   
                                 2                                      0.109375   
                                 3                                      0.500000   
<1940                            1                                      0.161243   
                                 2                                      0.167607   
                                 3                                      0.153153   

build_existing_model.geometry_foundation_type                           Heated Basement  \
build_existing_model.vintage_acs build_existing_model.geometry_stories                    
1940-59                          1                                             0.428950   
                                 2                                             0.401400   
                                 3                                             0.382353   
1960-79                          1                                             0.353704   
                                 2                                             0.390155   
                                 3                                             0.328358   
1980-99                          1                                             0.321059   
                                 2                                             0.322667   
                                 3                                             0.344828   
2000-09                          1                                             0.528662   
                                 2                                             0.586124   
                                 3                                             0.400000   
2010s                            1                                             0.595238   
                                 2                                             0.515625   
                                 3                                             0.500000   
<1940                            1                                             0.284024   
                                 2                                             0.277898   
                                 3                                             0.234234   

build_existing_model.geometry_foundation_type                           Pier and Beam  \
build_existing_model.vintage_acs build_existing_model.geometry_stories                  
1940-59                          1                                           0.014933   
                                 2                                           0.015169   
                                 3                                           0.029412   
1960-79                          1                   

In [98]:
### get foundation type distr by building type, vintage (EE prototype)

res_found_distr = res[res['build_existing_model.geometry_building_type_recs'] =="Single-Family Detached"].groupby(
    ['vintage_ee', 'stories',
    'build_existing_model.geometry_foundation_type'])['building_id'].count().to_frame()

# normalize
res_found_distr = res_found_distr.groupby(level = [0,1])['building_id'].apply(lambda x: x/x.sum()).unstack(level=2)
res_found_distr


build_existing_model.geometry_foundation_type     Crawl  Heated Basement  \
vintage_ee stories                                                         
1942-1978  2 or more stories                   0.146640         0.395112   
           less than 2 stories                 0.140828         0.395969   
post-1978  2 or more stories                   0.164179         0.420267   
           less than 2 stories                 0.168980         0.383673   
pre-1942   2 or more stories                   0.166943         0.275891   
           less than 2 stories                 0.161243         0.284024   

build_existing_model.geometry_foundation_type  Pier and Beam      Slab  \
vintage_ee stories                                                       
1942-1978  2 or more stories                        0.011541  0.209776   
           less than 2 stories                      0.011499  0.211310   
post-1978  2 or more stories                        0.003142  0.238806   
           less than 2 stories                      0.004898  0.267755   
pre-1942   2 or more stories                        0.010771  0.078708   
           less than 2 stories                      0.008876  0.063609   

build_existing_model.geometry_foundation_type  Unheated Basement  
vintage_ee stories                                                
1942-1978  2 or more stories                            0.236931  
           less than 2 stories                          0.240395  
post-1978  2 or more stories                            0.173606  
           less than 2 stories                          0.174694  
pre-1942   2 or more stories                            0.467688  
           less than 2 stories                          0.482249

In [168]:
res[res['build_existing_model.geometry_building_type_recs'] == "Single-Family Detached"].groupby([
    'build_existing_model.geometry_floor_area_bin','build_existing_model.geometry_wall_type',
'build_existing_model.vintage_acs'])['building_id'].count()

build_existing_model.geometry_floor_area_bin  build_existing_model.geometry_wall_type  build_existing_model.vintage_acs
0-1499                                        Masonry                                  1940-59                             1748
                                                                                       1960-79                              674
                                                                                       1980-99                               99
                                                                                       2000-09                               31
                                                                                       2010s                                  9
                                                                                       <1940                                681
                                              WoodStud                                 1940-59                  

In [171]:
res[res['build_existing_model.geometry_building_type_recs'] == "Single-Family Detached"].groupby([
    'build_existing_model.geometry_wall_type','stories',
'build_existing_model.vintage'])['gas_eui'].mean().unstack(level=-1)

build_existing_model.vintage                                    1940s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    1.270198   
                                        less than 2 stories  1.007639   
WoodStud                                2 or more stories    1.465058   
                                        less than 2 stories  1.189460   

build_existing_model.vintage                                    1950s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.741626   
                                        less than 2 stories  0.724631   
WoodStud                                2 or more stories    0.818172   
                                        less than 2 stories  0.805219   

build_existing_model.vintage                                    1960s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.724878   
                                        less than 2 stories  0.700151   
WoodStud                                2 or more stories    0.736374   
                                        less than 2 stories  0.745933   

build_existing_model.vintage                                    1970s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.725317   
                                        less than 2 stories  0.676803   
WoodStud                                2 or more stories    0.784502   
                                        less than 2 stories  0.714675   

build_existing_model.vintage                                    1980s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.487422   
                                        less than 2 stories  0.503628   
WoodStud                                2 or more stories    0.507886   
                                        less than 2 stories  0.528930   

build_existing_model.vintage                                    1990s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.471512   
                                        less than 2 stories  0.483470   
WoodStud                                2 or more stories    0.505770   
                                        less than 2 stories  0.484448   

build_existing_model.vintage                                    2000s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.342833   
                                        less than 2 stories  0.368342   
WoodStud                                2 or more stories    0.391012   
                                        less than 2 stories  0.415581   

build_existing_model.vintage                                    2010s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.271276   
                                        less than 2 stories  0.299280   
WoodStud                                2 or more stories    0.328131   
                                        less than 2 stories  0.392725   

build_existing_model.vintage                                    <1940  
build_existing_model.geometry_wall_type stories                        
Masonry                                 2 or more stories    0.875519  
                                        less than 2 stories  0.970570  
WoodStud                                2 or more stories    0.899317  
                                        less than 2 stories  1.028988

In [178]:
res[res['build_existing_model.geometry_building_type_recs'] == "Single-Family Detached"].groupby([
    'build_existing_model.geometry_wall_type','stories',
'build_existing_model.vintage'])['gas_eui'].mean().unstack(level=-1)

build_existing_model.vintage                                    1940s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.771525   
                                        less than 2 stories  0.684101   
WoodStud                                2 or more stories    0.852331   
                                        less than 2 stories  0.761761   

build_existing_model.vintage                                    1950s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.807702   
                                        less than 2 stories  0.690659   
WoodStud                                2 or more stories    0.898863   
                                        less than 2 stories  0.778996   

build_existing_model.vintage                                    1960s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.716290   
                                        less than 2 stories  0.622227   
WoodStud                                2 or more stories    0.811908   
                                        less than 2 stories  0.671872   

build_existing_model.vintage                                    1970s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.615688   
                                        less than 2 stories  0.557976   
WoodStud                                2 or more stories    0.621496   
                                        less than 2 stories  0.558558   

build_existing_model.vintage                                    1980s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.437891   
                                        less than 2 stories  0.456554   
WoodStud                                2 or more stories    0.468157   
                                        less than 2 stories  0.426862   

build_existing_model.vintage                                    1990s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.372117   
                                        less than 2 stories  0.371588   
WoodStud                                2 or more stories    0.385075   
                                        less than 2 stories  0.378791   

build_existing_model.vintage                                    2000s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.298608   
                                        less than 2 stories  0.295283   
WoodStud                                2 or more stories    0.310635   
                                        less than 2 stories  0.316522   

build_existing_model.vintage                                    2010s  \
build_existing_model.geometry_wall_type stories                         
Masonry                                 2 or more stories    0.294071   
                                        less than 2 stories  0.305492   
WoodStud                                2 or more stories    0.285653   
                                        less than 2 stories  0.271593   

build_existing_model.vintage                                    <1940  
build_existing_model.geometry_wall_type stories                        
Masonry                                 2 or more stories    0.744136  
                                        less than 2 stories  0.748823  
WoodStud                                2 or more stories    0.863646  
                                        less than 2 stories  0.849221

In [157]:
res_by_foundation = res[(res['build_existing_model.geometry_building_type_recs'] == "Single-Family Detached")].groupby([
    'vintage_ee', 'stories', 'build_existing_model.geometry_wall_type',
    'build_existing_model.geometry_foundation_type'
])
res_by_foundation1 = res_by_foundation['gas_eui'].mean().unstack(level=0)
res_by_foundation2 = res_by_foundation[
    'simulation_output_report.total_site_natural_gas_therm'].mean().unstack(level=0)
res_by_foundation3 = res_by_foundation['building_id'].count().unstack(level=0)
res_by_foundation4 = res_by_foundation['sqft'].mean().unstack(level=0)

res_by_foundation1['type'] = 'gas eui'
res_by_foundation2['type'] = 'gas'
res_by_foundation3['type'] = 'count'
res_by_foundation4['type'] = 'sqft'

res_by_foundation_all = pd.concat([res_by_foundation1,res_by_foundation2,res_by_foundation3, res_by_foundation4], axis=0)
#res_by_foundation_all.to_csv(os.path.join(result_path, iteration_path, 'building_type_vintage_foundation.csv'), index=True)
print('foundation type eui, sqft, count distr by vintage & bldg type exported')

res_by_foundation_all.head(10*4)

foundation type eui, sqft, count distr by vintage & bldg type exported


vintage_ee                                                                                                   1942-1978  \
stories             build_existing_model.geometry_wall_type build_existing_model.geometry_foundation_type                
2 or more stories   Masonry                                 Crawl                                             0.946397   
                                                            Heated Basement                                   0.701500   
                                                            Pier and Beam                                     1.460932   
                                                            Slab                                              0.909385   
                                                            Unheated Basement                                 0.957024   
                    WoodStud                                Crawl                                             1.034803   
                                                            Heated Basement                                   0.737714   
                                                            Pier and Beam                                     1.205539   
                                                            Slab                                              0.894523   
                                                            Unheated Basement                                 0.931147   
less than 2 stories Masonry                                 Crawl                                             1.011329   
                                                            Heated Basement                                   0.569086   
                                                            Pier and Beam                                     1.122423   
                                                            Slab                                              0.831922   
                                                            Unheated Basement                                 0.907919   
                    WoodStud                                Crawl                                             1.025597   
                                                            Heated Basement                                   0.580865   
                                                            Pier and Beam                                     1.437155   
                                                            Slab                                              0.815841   
                                                            Unheated Basement                                 0.952712   
2 or more stories   Masonry                                 Crawl                                          2107.981611   
                                                            Heated Basement                                1477.817918   
                                                            Pier and Beam                                  2342.787757   
                                                            Slab                                           1916.769546   
                                                            Unheated Basement                              1901.188474   
                    WoodStud                                Crawl                                          2127.679426   
                                                            Heated Basement                                1488.953172   
                                                            Pier and Beam                                  2052.152961   
                                                            Slab                                           1876.137204   
                                                            Unheated Basement                              1818.131425   
less than 2 stories Masonry                                 Crawl                               

In [165]:
vintage = ['2010s'] # <--- need to remove/change lines with arrows
res2 = res[res['build_existing_model.vintage'].isin(vintage)] # <--- need to remove/change lines with arrows

### Get results by building prototypes from Elevate Energy
#### Note: cannot add prototype tags to ResStock results as prototypes overlap in chars

In [159]:
## Elevate Energy prototype chars
groups = pd.read_csv(os.path.join(result_path,'Groups.csv'))
print('prototypes loaded')


prototypes loaded


In [166]:
def get_res_by_prototype(res, row):
    res_group_i = res[res['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        res['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        res['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        res['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]
    
    return res_group_i

Metric_map1 = {'count': 'completed_status',
           'median gas': 'simulation_output_report.total_site_natural_gas_therm',
           'median elec': 'simulation_output_report.total_site_electricity_kwh'}
Metric_map2 = {'mean gas': 'simulation_output_report.total_site_natural_gas_therm',
           'mean elec': 'simulation_output_report.total_site_electricity_kwh'} # metric: res_var

## initialize
for metric in {**Metric_map1, **Metric_map2}:
    groups[metric] = np.NAN

### (1) get median consumption
for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res2, row) # <----
    
    for metric, res_var in Metric_map1.items():
        if metric == 'count':
            groups.iloc[i, groups.columns.get_loc('count')] = \
                len(res_group_i[res_group_i['completed_status']=="Success"])
        else:
            groups.iloc[i, groups.columns.get_loc(metric)] = \
                res_group_i[res_var].median()

groups['Gas Diff-med(%)'] = ((groups['Non-normalized gas usage'] - groups['median gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-med(%)'] = ((groups['Non-normalized elec. usage'] - groups['median elec']) / groups['Non-normalized elec. usage']) * 100
print('>> median consumption computed')

### (2) get mean consumption
for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res2, row) # <----
    
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

groups['Gas Diff-mean(%)'] = ((groups['Non-normalized gas usage'] - groups['mean gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-mean(%)'] = ((groups['Non-normalized elec. usage'] - groups['mean elec']) / groups['Non-normalized elec. usage']) * 100
print('>> mean consumption computed')


>> median consumption computed
>> mean consumption computed


In [167]:
### (3) get median & mean sqft & eui
Metric_map1 = {'median sqft': 'sqft',
               'median gas eui': 'gas_eui', 
               'median elec eui': 'elec_eui'}
Metric_map2 = {'mean sqft': 'sqft', 
               'mean gas eui': 'gas_eui', 
               'mean elec eui': 'elec_eui'}

## initialize
for metric in {**Metric_map1, **Metric_map2}:
    groups[metric] = np.NAN


for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res2, row) # <----
    
    ### (3.1) get median values
    for metric, res_var in Metric_map1.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].median()

    ### (3.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

print('>> median & mean sqft & eui computed:'); print(f'for {vintage}:')

groups.set_index(['HousingGroupNo','HousingGroupName'])[['median sqft', 'mean sqft',
                                                         'median gas','mean gas',
                                                         'median gas eui', 'mean gas eui', 
                                                         'median elec','mean elec', 
                                                         'median elec eui','mean elec eui']]



>> median & mean sqft & eui computed:
for ['2010s']:


,,median sqft,mean sqft,median gas,mean gas,median gas eui,mean gas eui,median elec,mean elec,median elec eui,mean elec eui
HousingGroupNo,HousingGroupName,,,,,,,,,,
1,Masonry Post-1978 less than 2 stories,2663.0,3239.296296,685.569748,781.744899,0.325663,0.299280,10666.754586,12712.324541,4.298968,5.103490
2,Masonry All Years Split Level,3301.0,4326.293103,858.785670,1001.233269,0.304703,0.284313,12090.559098,15838.690761,3.616753,4.383149
3,Masonry Post-1978 2 or more stories,3301.0,5273.032258,1015.241913,1192.400559,0.267501,0.271276,13720.177184,18561.654888,2.948503,3.755756
4,Masonry 1942-1978 Less than 2 Stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Masonry 1942-1978 2 or more stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Masonry Pre-1942 less than 2 stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Masonry Pre-1942 2 or more stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Frame Post-1978 Less than 2 stories,1933.0,2868.937500,804.298401,1025.723760,0.391052,0.392725,9299.957309,10139.928000,3.651281,4.595484
9,Frame Post-1978 Split Level,3301.0,4248.620000,1091.458889,1311.109210,0.353135,0.348801,12015.386091,16146.406903,3.417567,4.590739


In [164]:
### (3) get median & mean sqft & eui
Metric_map1 = {'median sqft': 'sqft',
               'median gas eui': 'gas_eui', 
               'median elec eui': 'elec_eui'}
Metric_map2 = {'mean sqft': 'sqft', 
               'mean gas eui': 'gas_eui', 
               'mean elec eui': 'elec_eui'}

## initialize
for metric in {**Metric_map1, **Metric_map2}:
    groups[metric] = np.NAN


for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res2, row) # <----
    
    ### (3.1) get median values
    for metric, res_var in Metric_map1.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].median()

    ### (3.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

print('>> median & mean sqft & eui computed:'); print(f'for {vintage}:')

groups.set_index(['HousingGroupNo','HousingGroupName'])[['median sqft', 'mean sqft',
                                                         'median gas','mean gas',
                                                         'median gas eui', 'mean gas eui', 
                                                         'median elec','mean elec', 
                                                         'median elec eui','mean elec eui']]

>> median & mean sqft & eui computed:
for ['2000s']:


,,median sqft,mean sqft,median gas,mean gas,median gas eui,mean gas eui,median elec,mean elec,median elec eui,mean elec eui
HousingGroupNo,HousingGroupName,,,,,,,,,,
1,Masonry Post-1978 less than 2 stories,2176.0,2783.568075,662.383338,800.747613,0.312326,0.368342,8929.921520,10304.484231,3.904222,4.859313
2,Masonry All Years Split Level,2663.0,3573.011655,883.922929,1031.056738,0.320624,0.355498,10052.960721,12362.308686,3.546154,4.356955
3,Masonry Post-1978 2 or more stories,3301.0,4351.490741,1105.983975,1258.167125,0.323061,0.342833,11759.536149,14391.552246,3.247697,3.861574
4,Masonry 1942-1978 Less than 2 Stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Masonry 1942-1978 2 or more stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Masonry Pre-1942 less than 2 stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Masonry Pre-1942 2 or more stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Frame Post-1978 Less than 2 stories,2176.0,2550.178571,766.604939,829.337051,0.374676,0.415581,8840.154313,10890.316820,4.249133,5.318390
9,Frame Post-1978 Split Level,2663.0,3597.288889,1041.417938,1202.840019,0.376675,0.399748,10037.846304,12468.782812,3.564338,4.294541


In [ ]:
##### (4) get heating and cooling end uses to "groups"
Metric_map1 = ['frac_gas_heating', 'frac_elec_heating', 'frac_elec_cooling']
Metric_map2 = {'size_heating_kbut_h': 'simulation_output_report.size_heating_system_kbtu_h',
               'size_cooling_kbut_h': 'simulation_output_report.size_cooling_system_kbtu_h'}

## initialize
for metric in Metric_map1 + list(Metric_map2.keys()):
    groups[metric] = np.NAN

for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    ### (4.1) get fractions
    groups.iloc[i, groups.columns.get_loc('frac_gas_heating')] = \
        res_group_i['total_gas_heating'].mean()/res_group_i['simulation_output_report.total_site_natural_gas_therm'].mean()

    groups.iloc[i, groups.columns.get_loc('frac_elec_heating')] = \
        res_group_i['total_elec_heating'].mean()/res_group_i['simulation_output_report.total_site_electricity_kwh'].mean()
    
    groups.iloc[i, groups.columns.get_loc('frac_elec_cooling')] = \
        res_group_i['total_elec_cooling'].mean()/res_group_i['simulation_output_report.total_site_electricity_kwh'].mean()
    
    ### (4.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

print('computed')

In [52]:
### (5) get heating & cooling and floor area char breakdowns

House = []
House_chars = [cooling_col, heating_col,
              #'build_existing_model.geometry_floor_area',
              'build_existing_model.insulation_unfinished_attic']

for i, row in groups.iterrows():
    
    res_group_i = get_res_by_prototype(res, row)
    
    Ns = len(res_group_i.query('completed_status=="Success"'))
    Hi = []
    for char in House_chars:
        Nchar = res_group_i.groupby(char)['building_id'].count()/Ns # fractions
        Nchar = Nchar.rename('fraction').to_frame()
        Nchar['housing_char'] = Nchar.index.name
        Hi.append(Nchar)
    
    Hi = pd.concat(Hi, axis=0)
    Hi.index.name = 'sub_char'
    Hi = Hi.reset_index()
    Hi['HousingGroupNo'] = row['HousingGroupNo']
    
    House.append(Hi)
    
House = pd.concat(House, axis=0)
House = pd.pivot_table(House, values='fraction', index=['HousingGroupNo'],
                    columns=['housing_char','sub_char'], aggfunc=np.sum)

print('computed')

computed


In [53]:
## Combine "groups" + "House" into "Out" for export
out = groups.drop(['Stories','WallType','Vintage'], axis=1)
out = pd.concat([out.set_index('HousingGroupNo')], axis=1, keys=['none']).join(House)
out

none  \
                                     HousingGroupName   
HousingGroupNo                                          
1               Masonry Post-1978 less than 2 stories   
2                       Masonry All Years Split Level   
3                 Masonry Post-1978 2 or more stories   
4               Masonry 1942-1978 Less than 2 Stories   
5                 Masonry 1942-1978 2 or more stories   
6                Masonry Pre-1942 less than 2 stories   
7                  Masonry Pre-1942 2 or more stories   
8                 Frame Post-1978 Less than 2 stories   
9                         Frame Post-1978 Split Level   
10                  Frame Post-1978 2 or more stories   
11                Frame 1942-1978 less than 2 stories   
12                         Frame Pre-1942 Split Level   
13                  Frame 1942-1978 2 or more stories   
14                 Frame Pre-1942 less than 2 stories   
15                   Frame Pre-1942 2 or more stories   

                                                                            \
               Non-normalized gas usage Non-normalized elec. usage   count   
HousingGroupNo                                                               
1                                  1133                      10282   697.0   
2                                  1416                       9661  8897.0   
3                                  1812                      18225   525.0   
4                                  1210                       8779  4826.0   
5                                  1427                      10549  1182.0   
6                                  1504                       9299   732.0   
7                                  2044                      13272   935.0   
8                                  1054                       9353   562.0   
9                                  1168                      10445  1284.0   
10                                 1422                      13495   722.0   
11                                 1176                       9213  2820.0   
12                                 1297                       9964  2102.0   
13                                 1530                      12543  1504.0   
14                                 1340                       9103   698.0   
15                                 1753                      11967  1404.0   

                                                                      \
                 median gas   median elec     mean gas     mean elec   
HousingGroupNo                                                         
1                827.585075   8833.301226   893.232987   9960.114058   
2               1184.916074   9266.321551  1308.704726  10773.268192   
3               1186.896059  11504.558180  1354.705805  13962.779918   
4               1066.568715   8673.199620  1144.934814   9750.833165   
5               1585.913502  10468.396412  1740.809866  12129.312601   
6               1243.747079   9165.958694  1326.414207  10406.407874   
7               1727.143977  11221.616930  1877.769129  13438.769240   
8                857.235591   8833.450771   909.545470  10121.030958   
9               1081.240703   9922.163042  1213.536170  11898.767307   
10              1250.522248  10769.909253  1450.160509  13282.545462   
11              1138.183147   8783.392501  1202.352338   9772.250161   
12              1547.068005  10397.400468  1652.098364  11851.278133   
13              1589.492116  10525.777462  1765.290380  11955.967475   
14              1262.042987   8757.546055  1335.188968   9886.813252   
15              1694.297166  11104.625516  1809.650186  12827.913807   

                                                 ...  \
               Gas Diff-med(%) Elec Diff-med(%)  ...   
HousingGroupNo                                   ...   
1                    26.956304        14.089659  ...   
2                    16.319486         4.085275  ...   
3                    34.498010        36

### Write output to folder

In [54]:
#out_name = "Groups_results" + "_" + baseline_result.split("_")[2]
out.to_csv(os.path.join(result_path, iteration_path, 'baseline_prototype_results.csv'), index=True)
print(f'file saved to:\n  {os.path.join(result_path, iteration_path)}')

file saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_2007_10/baseline
